# Import Model

In [1]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1)
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.act1  = nn.SiLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(out_channels)
        self.shortcut = None
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        self.act2 = nn.SiLU()

    def forward(self, x):
        identity = x
        out = self.act1(self.bn1(self.conv1(x)))
        out = self.act2(self.bn2(self.conv2(out)))
        if self.shortcut is not None:
            identity = self.shortcut(x)
        return out + identity



class DeepTileEncoder(nn.Module):
    """加深的 Tile 分支：全局信息，多尺度池化 + 三层 MLP"""
    def __init__(self, out_dim, in_channels=3, negative_slope=0.01):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.MaxPool2d(2)  # 78→39
        )
        self.layer1 = nn.Sequential(
            ResidualBlock(32, 64),
            ResidualBlock(64, 64),
            nn.MaxPool2d(2)  # 39→19
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128),
            ResidualBlock(128, 128),
            nn.MaxPool2d(2)  # 19→9
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(128, 256),
            ResidualBlock(256, 256)
        )  # 保持 9×9

        # 多尺度池化
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # [B,256,1,1]
        self.mid_pool    = nn.AdaptiveAvgPool2d((3, 3))  # [B,256,3,3]

        total_dim = 256*1*1 + 256*3*3
        # 三层 MLP：total_dim → 2*out_dim → out_dim → out_dim
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.Linear(total_dim, out_dim*4),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(out_dim*4, out_dim*2),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(out_dim*2, out_dim),
            nn.LeakyReLU(negative_slope),
        )

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x: [B,256,9,9]
        g = self.global_pool(x).contiguous().reshape(x.size(0), -1)  # [B,256]
        m = self.mid_pool(x).contiguous().reshape(x.size(0), -1)     # [B,256*3*3]

        return self.fc(torch.cat([g, m], dim=1))


class SubtileEncoder(nn.Module):
    """多尺度 Subtile 分支：局部信息 + 两层 MLP"""
    def __init__(self, out_dim, in_channels=3, negative_slope=0.01):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.MaxPool2d(2)  # 26→13
        )
        self.layer1 = nn.Sequential(
            ResidualBlock(32, 64),
            ResidualBlock(64, 64),
            nn.MaxPool2d(2)  # 13→6
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128),
            ResidualBlock(128, 128)
        )  # 保持 6×6

        self.global_pool = nn.AdaptiveAvgPool2d((1,1))
        self.mid_pool    = nn.AdaptiveAvgPool2d((2,2))
        self.large_pool    = nn.AdaptiveAvgPool2d((3,3))

        total_dim = 128*1*1 + 128*2*2 + 128*3*3
        # 两层 MLP：total_dim → out_dim*2 → out_dim
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.Linear(total_dim, out_dim*2),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(out_dim*2, out_dim),
            nn.LeakyReLU(negative_slope),
        )

    def forward(self, x):
        B, N, C, H, W = x.shape
        x = x.contiguous().reshape(B*N, C, H, W)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        # g,m: [B*N, feat]
        g = self.global_pool(x).contiguous().reshape(B, N, -1)
        m = self.mid_pool(x).contiguous().reshape(B, N, -1)
        l = self.large_pool(x).contiguous().reshape(B, N, -1)

        # 合并 N 张 subtiles，再 FC
        feat = torch.cat([g, m, l], dim=2).mean(dim=1).contiguous()  # [B, total_dim]
        return self.fc(feat)
class CenterSubtileEncoder(nn.Module):
    """專門處理中心 subtile 的 Encoder"""
    def __init__(self, out_dim, in_channels=3, negative_slope= 0.01):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.MaxPool2d(2)  # 26→13
        )
        self.layer1 = nn.Sequential(
            ResidualBlock(32, 64),
            ResidualBlock(64, 64),
            nn.MaxPool2d(2)  # 13→6
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128),
            ResidualBlock(128, 128)
        )  # 6×6

        # 多尺度池化
        self.global_pool = nn.AdaptiveAvgPool2d((1,1))
        self.mid_pool    = nn.AdaptiveAvgPool2d((2,2))
        self.large_pool    = nn.AdaptiveAvgPool2d((3,3))

        total_dim = 128*1*1 + 128*2*2 + 128*3*3
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.Linear(total_dim, out_dim*2),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(out_dim*2, out_dim),
            nn.LeakyReLU(negative_slope),
        )

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        g = self.global_pool(x).contiguous().reshape(x.size(0), -1)
        m = self.mid_pool(x).contiguous().reshape(x.size(0), -1)
        l = self.large_pool(x).contiguous().reshape(x.size(0), -1)

        return self.fc(torch.cat([g, m, l], dim=1)).contiguous()



class VisionMLP_MultiTask(nn.Module):
    """整體多任務模型：融合 tile + subtile + center，使用動態權重融合"""
    def __init__(self, tile_dim=128, subtile_dim=64, output_dim=35, negative_slope=0.01):
        super().__init__()
        self.encoder_tile    = DeepTileEncoder(tile_dim)
        self.encoder_subtile = SubtileEncoder(subtile_dim)
        self.encoder_center  = CenterSubtileEncoder(subtile_dim)

        # 輸出 decoder：輸入為 tile_dim (因為融合後只剩一個 vector)
        self.decoder = nn.Sequential(
            nn.Linear(tile_dim + subtile_dim + subtile_dim , 256),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(64, output_dim),
        )

    def forward(self, tile, subtiles):
        tile = tile.contiguous()
        subtiles = subtiles.contiguous()
        center = subtiles[:, 4]

        f_tile = self.encoder_tile(tile)         # [B, tile_dim]
        f_sub  = self.encoder_subtile(subtiles)  # [B, subtile_dim]
        f_center = self.encoder_center(center)   # [B, subtile_dim]

        # 拼接三個分支做 gating
        features_cat = torch.cat([f_tile, f_sub, f_center], dim=1)  # [B, tile+sub+center]
        return self.decoder(features_cat)





# 用法示例
model = VisionMLP_MultiTask(tile_dim=128, subtile_dim=128, output_dim=35)


# —— 5) 确保只有 decoder 可训练 ——  
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable / total params = {trainable:,} / {total:,}")

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable / total params = {trainable:,} / {total:,}")
model


Trainable / total params = 6,679,843 / 6,679,843
Trainable / total params = 6,679,843 / 6,679,843


VisionMLP_MultiTask(
  (encoder_tile): DeepTileEncoder(
    (layer0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        

## Load Model

# Import training data

## Same in multiple .pt

In [2]:
import os
import torch
import random
import inspect
from python_scripts.import_data import load_all_tile_data

# 用法範例
#folder = "dataset/spot-rank/version-3/only_tile_sub/original_train"
folder = "dataset/spot-rank/filtered_directly_rank/masked/realign/Macenko_4_7_masked/filtered/train_data/"

grouped_data = load_all_tile_data( 
        folder_path=folder,
        model=model,
        fraction=1,
        shuffle=False
    )

    # grouped_data 現在只會有 model.forward() 需要的 key，
    # 像 ['tile','subtiles','neighbors','norm_coord','node_feat','adj_list','edge_feat','label','source_idx']
print("Loaded keys:", grouped_data.keys())
print("Samples:", len(next(iter(grouped_data.values()))))




/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/import_data.py:251: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_

Loaded keys: dict_keys(['slide_idx', 'position', 'subtiles', 'label', 'tile', 'source_idx'])
Samples: 8348


In [3]:
from python_scripts.import_data import convert_item, get_model_inputs

import torch
from torch.utils.data import Dataset
import inspect
import numpy as np

class importDataset(Dataset):
    def __init__(self, data_dict, model, image_keys=None, transform=None, print_sig=False):
        self.data = data_dict
        self.image_keys = set(image_keys) if image_keys is not None else set()
        self.transform = transform if transform is not None else lambda x: x
        self.forward_keys = list(get_model_inputs(model, print_sig=print_sig).parameters.keys())

        expected_length = None
        for key, value in self.data.items():
            if expected_length is None:
                expected_length = len(value)
            if len(value) != expected_length:
                raise ValueError(f"資料欄位 '{key}' 的長度 ({len(value)}) 與預期 ({expected_length}) 不一致。")

        for key in self.forward_keys:
            if key not in self.data:
                raise ValueError(f"data_dict 缺少模型 forward 所需欄位: '{key}'。目前可用的欄位: {list(self.data.keys())}")
        if "label" not in self.data:
            raise ValueError(f"data_dict 必須包含 'label' 欄位。可用的欄位: {list(self.data.keys())}")
        if "source_idx" not in self.data:
            raise ValueError("data_dict 必須包含 'source_idx' 欄位，用於 trace 原始順序對應。")
        if "position" not in self.data:
            raise ValueError("data_dict 必須包含 'position' 欄位，用於 trace 原始順序對應。")
    def __len__(self):
        return len(next(iter(self.data.values())))

    def __getitem__(self, idx):
        sample = {}
        for key in self.forward_keys:
            value = self.data[key][idx]
            value = self.transform(value)
            value = convert_item(value, is_image=(key in self.image_keys))
            if isinstance(value, torch.Tensor):
                value = value.float()
            sample[key] = value

        label = self.transform(self.data["label"][idx])
        label = convert_item(label, is_image=False)
        if isinstance(label, torch.Tensor):
            label = label.float()
        sample["label"] = label

        # 加入 source_idx
        source_idx = self.data["source_idx"][idx]
        sample["source_idx"] = torch.tensor(source_idx, dtype=torch.long)
        # 加入 position （假设 data_dict 中 'position' 是 (x, y) 或 [x, y]）
        pos = self.data["position"][idx]
        sample["position"] = torch.tensor(pos, dtype=torch.float)
        return sample
    def check_item(self, idx=0, num_lines=5):
        expected_keys = self.forward_keys + ['label', 'source_idx', 'position']
        sample = self[idx]
        print(f"🔍 Checking dataset sample: {idx}")
        for key in expected_keys:
            if key not in sample:
                print(f"❌ 資料中缺少 key: {key}")
                continue
            tensor = sample[key]
            if isinstance(tensor, torch.Tensor):
                try:
                    shape = tensor.shape
                except Exception:
                    shape = "N/A"
                dtype = tensor.dtype if hasattr(tensor, "dtype") else "N/A"
                output_str = f"📏 {key} shape: {shape} | dtype: {dtype}"
                if tensor.numel() > 0:
                    try:
                        tensor_float = tensor.float()
                        mn = tensor_float.min().item()
                        mx = tensor_float.max().item()
                        mean = tensor_float.mean().item()
                        std = tensor_float.std().item()
                        output_str += f" | min: {mn:.3f}, max: {mx:.3f}, mean: {mean:.3f}, std: {std:.3f}"
                    except Exception:
                        output_str += " | 無法計算統計數據"
                print(output_str)
                if key not in self.image_keys:
                    if tensor.ndim == 0:
                        print(f"--- {key} 資料為純量:", tensor)
                    elif tensor.ndim == 1:
                        print(f"--- {key} head (前 {num_lines} 個元素):")
                        print(tensor[:num_lines])
                    else:
                        print(f"--- {key} head (前 {num_lines} 列):")
                        print(tensor[:num_lines])
            else:
                # 如果 position 存的是 list/tuple/etc，也会走这里
                print(f"📏 {key} (非 tensor 資料):", tensor)
        print("✅ All checks passed!")


full_dataset = importDataset(grouped_data, model,
                             image_keys=['tile','subtiles'],
                             transform=lambda x: x)

full_dataset.check_item()

🔍 Checking dataset sample: 0
📏 tile shape: torch.Size([3, 78, 78]) | dtype: torch.float32 | min: 0.129, max: 1.000, mean: 0.653, std: 0.150
📏 subtiles shape: torch.Size([9, 3, 26, 26]) | dtype: torch.float32 | min: 0.129, max: 1.000, mean: 0.653, std: 0.150
📏 label shape: torch.Size([35]) | dtype: torch.float32 | min: 1.000, max: 35.000, mean: 18.000, std: 10.247
--- label head (前 5 個元素):
tensor([12., 24., 18.,  6., 30.])
📏 source_idx shape: torch.Size([]) | dtype: torch.int64 | min: 0.000, max: 0.000, mean: 0.000, std: nan
--- source_idx 資料為純量: tensor(0)
📏 position shape: torch.Size([2]) | dtype: torch.float32 | min: 0.171, max: 0.632, mean: 0.401, std: 0.326
--- position head (前 5 個元素):
tensor([0.6318, 0.1707])
✅ All checks passed!


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_93313/3062057575.py:79: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1744320376245/work/aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = tensor_float.std().item()


In [43]:

from python_scripts.prediction_features_individual import  *
import numpy as np
from torch.utils.data import DataLoader

def generate_meta_features(oof_preds: np.ndarray, cell_idx: int):
    """
    Generate meta‐features for a single cell type, using all self-centric functions.

    Parameters
    ----------
    oof_preds : np.ndarray, shape (n_samples, C)
        The OOF prediction matrix.
    cell_idx : int
        Index of the “self” cell type (0 ≤ cell_idx < C).

    Returns
    -------
    features : np.ndarray, shape (n_samples, n_meta)
    names    : list of str, length n_meta
    """
    outputs = []

    # 1) 原始 self 预测值
    pi = oof_preds[:, cell_idx:cell_idx+1]  # (n_samples, 1)
    raw_names = [f"oof_pred_{i}" for i in range(35)]
    outputs.append((oof_preds, raw_names))

    # 2) self vs others 差值
    # feats, names = compute_self_vs_others_diff(oof_preds, cell_idx)
    # outputs.append((feats, names))

    # # 3) self vs others 比值
    # feats, names = compute_self_vs_others_ratio(oof_preds, cell_idx)
    # outputs.append((feats, names))

    # # 4) 归一化排名
    # feats, names = compute_normalized_rank(oof_preds, cell_idx)
    # outputs.append((feats, names))

    # # 5) z-score
    # feats, names = compute_self_zscore(oof_preds, cell_idx)
    # outputs.append((feats, names))

    # # 6) 前后 k=3 的统计
    # feats, names = compute_self_topk_stats(oof_preds, cell_idx, k=3)
    # outputs.append((feats, names))

    # # 7) 与上下游邻近差分
    # feats, names = compute_self_adjacent_diffs(oof_preds, cell_idx)
    # outputs.append((feats, names))

    # # 8) 与上下游邻近比率
    # feats, names = compute_self_adjacent_ratio(oof_preds, cell_idx)
    # outputs.append((feats, names))

    # # 9) 排序后，self 与前后峰差
    # feats, names = compute_self_vs_rank_diff(oof_preds, cell_idx)
    # outputs.append((feats, names))

    # —— 解包 & 检查 —— 
    feat_list, name_seq = zip(*outputs)
    for feats, names_block in zip(feat_list, name_seq):
        # 一维也当成列向量
        ncols = feats.shape[1] if feats.ndim == 2 else 1
        if ncols != len(names_block):
            raise ValueError(
                f"Block {names_block[0]}: got {ncols} cols but {len(names_block)} names"
            )

    # 拼接
    features = np.concatenate(feat_list, axis=1)
    name_list = [n for block in name_seq for n in block]
    print(f"✅ Generated meta‐features for cell {cell_idx}, shape = {features.shape}")
    return features, name_list


In [6]:
from collections import defaultdict
import numpy as np

def diagnose_meta_nonfinite(meta: np.ndarray, names: list[str]):
    """
    按名字前缀分组，统计每组：
      - 原始特征数（列数）
      - 总值数（列数 × 行数）
      - NaN 值数量
      - ±Inf 值数量
      - 非数值（non-finite）总数

    Parameters
    ----------
    meta  : np.ndarray, shape (n_samples, n_features)
    names : list of str, length n_features

    Returns
    -------
    stats : dict[prefix, dict]  
        每个 prefix 对应一个字典，
        包含 'n_feats','total_vals','n_nan','n_inf','n_nonfinite'。
    """
    groups = defaultdict(list)
    # 按前缀分组
    for idx, nm in enumerate(names):
        prefix = nm.split('_', 1)[0]
        groups[prefix].append(idx)

    stats = {}
    for prefix, idxs in groups.items():
        sub = meta[:, idxs]  # shape (n_samples, n_group_feats)
        n_feats = sub.shape[1]
        total_vals = sub.size
        n_nan = np.isnan(sub).sum()
        n_inf = np.isinf(sub).sum()
        n_nonfinite = (~np.isfinite(sub)).sum()

        stats[prefix] = {
            'n_feats':        n_feats,
            'total_vals':     total_vals,
            'n_nan':          int(n_nan),
            'n_inf':          int(n_inf),
            'n_nonfinite':    int(n_nonfinite),
        }
        print(
            f"Group '{prefix}': "
            f"features={n_feats}, "
            f"values={total_vals}, "
            f"non-finite={n_nonfinite} "
            f"(nan={n_nan}, inf={n_inf})"
        )
    return stats


In [39]:
meta_i.shape

(2197, 35)

In [44]:
import os
import numpy as np
import joblib
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from scipy.stats import rankdata
from python_scripts.import_data import importDataset
from python_scripts.operate_model import predict
from lightgbm import early_stopping, log_evaluation
import h5py
import pandas as pd
from python_scripts.pretrain_model import PretrainedEncoderRegressor
# ---------------- Settings ----------------
trained_oof_model_folder = 'output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/'
n_folds    = len([d for d in os.listdir(trained_oof_model_folder) if d.startswith('fold')])
n_samples  = len(full_dataset)
C          = 35
BATCH_SIZE = 64
start_fold = 0

tile_dim = 128
center_dim = 128
neighbor_dim = 128
fusion_dim = tile_dim + center_dim + neighbor_dim

pretrained_ae_name = 'AE_Center_noaug'
pretrained_ae_path = f"AE_model/128/{pretrained_ae_name}/best.pt"
ae_type = 'center'

# Ground truth label (全 dataset)
y_true = np.vstack([ full_dataset[i]['label'].cpu().numpy() for i in range(n_samples) ])

# Build CV splitter (must match first stage splits)
logo = LeaveOneGroupOut()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

lgb_base  = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    learning_rate=0.03,       # 0.03~0.05 在 2k~5k 轮内通常收敛
    n_estimators=5000,        # 配合早停
    max_depth=6,              # 降低树深
    num_leaves=34,            # ≲2^6
    min_data_in_leaf=50,      # 每叶最少 50 样本
    colsample_bytree=0.7,     # 保留 70% 特征
    subsample=0.8,            # 保留 80% 样本
    subsample_freq=1,
    reg_alpha=1.0,            # L1 正则
    reg_lambda=1.0,           # L2 正则
    verbosity=-1
)


slide_idx = np.array(grouped_data['slide_idx'])   # shape (N,)


for fold_id, (tr_idx, va_idx) in enumerate(
    logo.split(X=np.zeros(n_samples), y=None, groups=slide_idx)):

    # if fold_id > start_fold:
    #     print(f"⏭️ Skipping fold {fold_id}")
    #     continue

    print(f"\n🚀 Starting fold {fold_id}...")
    ckpt_path = os.path.join(trained_oof_model_folder, f"fold{fold_id}", "best_model.pt")

    # === Load model and predict OOF ===
    net = VisionMLP_MultiTask(tile_dim=tile_dim, subtile_dim=center_dim, output_dim=C)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net = net.to(device).eval()

    val_ds = Subset(full_dataset, va_idx)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

    preds = []
    with torch.no_grad():
        for batch in val_loader:
            tiles = batch['tile'].to(device)
            subtiles = batch['subtiles'].to(device)
            center = subtiles[:, 4].contiguous()

            f_c = net.encoder_center(center)
            f_n = net.encoder_subtile(subtiles)
            f_t = net.encoder_tile(tiles)
            fuse = torch.cat([f_c, f_n, f_t], dim=1).contiguous()
            output = net.decoder(fuse)

            preds.append(output.cpu())

    preds = torch.cat(preds, dim=0).numpy()

    y_val      = y_true[va_idx]           # (n_val, 35)
 # --- 2) 训练 meta‐model: 每个 cell_type 单独用自己的特征去拟合残差 ---
    meta_model = MultiOutputRegressor(lgb_base)
    meta_model.estimators_ = []

    for i in range(C):
        # 2.1 生成第 i 类的“self vs others”特征
        meta_i, names_i = generate_meta_features(preds, i)
        # 简单检查
        diagnose_meta_nonfinite(meta_i, names_i)
        # 2.2 划分 train/val
        X_tr, X_va, y_tr, y_va = train_test_split(
            meta_i, y_val[:, i], test_size=0.2, random_state=42
        )

        # 2.3 训练单输出 LGBM
        print(f"  → Cell {i}: meta-feat dim={meta_i.shape[1]}")
        m = lgb.LGBMRegressor(**lgb_base.get_params())
        m.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            callbacks=[early_stopping(stopping_rounds=200), log_evaluation(period=100)]
        )
        meta_model.estimators_.append(m)

    # --- 3) 存模型 ---
    outp = os.path.join(trained_oof_model_folder, f"fold{fold_id}", "meta_model.pkl")
    joblib.dump(meta_model, outp)
    print(f"✅ Saved fold {fold_id} meta-model → {outp}")


🚀 Starting fold 0...


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_23908/861230578.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt_path, map

✅ Generated meta‐features for cell 0, shape = (2197, 35)
Group 'oof': features=35, values=76895, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 6.61282
[200]	valid_0's rmse: 6.66403
Early stopping, best iteration is:
[68]	valid_0's rmse: 6.60016
✅ Generated meta‐features for cell 1, shape = (2197, 35)
Group 'oof': features=35, values=76895, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 2.89674
[200]	valid_0's rmse: 2.91559
[300]	valid_0's rmse: 2.93534
Early stopping, best iteration is:
[111]	valid_0's rmse: 2.89341
✅ Generated meta‐features for cell 2, shape = (2197, 35)
Group 'oof': features=35, values=76895, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 4.59679
[200]	valid_0's rmse: 4.62433
[300]	valid_0

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_23908/861230578.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt_path, map

✅ Generated meta‐features for cell 0, shape = (2269, 35)
Group 'oof': features=35, values=79415, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 2.71752
[200]	valid_0's rmse: 2.73514
Early stopping, best iteration is:
[86]	valid_0's rmse: 2.71479
✅ Generated meta‐features for cell 1, shape = (2269, 35)
Group 'oof': features=35, values=79415, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 2.5502
[200]	valid_0's rmse: 2.5564
[300]	valid_0's rmse: 2.56002
Early stopping, best iteration is:
[134]	valid_0's rmse: 2.54598
✅ Generated meta‐features for cell 2, shape = (2269, 35)
Group 'oof': features=35, values=79415, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 2.21726
[200]	valid_0's rmse: 2.2208
Early stopping, 

...

✅ Generated meta‐features for cell 0, shape = (1187, 35)
Group 'oof': features=35, values=41545, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 6.72485
[200]	valid_0's rmse: 6.7875
Early stopping, best iteration is:
[22]	valid_0's rmse: 6.67087
✅ Generated meta‐features for cell 1, shape = (1187, 35)
Group 'oof': features=35, values=41545, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 3.02651
[200]	valid_0's rmse: 3.01474
[300]	valid_0's rmse: 3.03753
Early stopping, best iteration is:
[169]	valid_0's rmse: 3.01024
✅ Generated meta‐features for cell 2, shape = (1187, 35)
Group 'oof': features=35, values=41545, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 3.93007
[200]	valid_0's rmse: 3.95104
Early stopping

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_23908/861230578.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt_path, map

✅ Generated meta‐features for cell 0, shape = (1677, 35)
Group 'oof': features=35, values=58695, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 5.90573
[200]	valid_0's rmse: 5.90629
[300]	valid_0's rmse: 5.92101
Early stopping, best iteration is:
[131]	valid_0's rmse: 5.89693
✅ Generated meta‐features for cell 1, shape = (1677, 35)
Group 'oof': features=35, values=58695, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 2.12603
[200]	valid_0's rmse: 2.12522
[300]	valid_0's rmse: 2.13629
Early stopping, best iteration is:
[130]	valid_0's rmse: 2.11497
✅ Generated meta‐features for cell 2, shape = (1677, 35)
Group 'oof': features=35, values=58695, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 4.03768
[200]	valid_

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_23908/861230578.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt_path, map

✅ Generated meta‐features for cell 0, shape = (328, 35)
Group 'oof': features=35, values=11480, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 6.10181
[200]	valid_0's rmse: 6.21846
Early stopping, best iteration is:
[37]	valid_0's rmse: 6.05314
✅ Generated meta‐features for cell 1, shape = (328, 35)
Group 'oof': features=35, values=11480, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 1.33535
[200]	valid_0's rmse: 1.34848
Early stopping, best iteration is:
[23]	valid_0's rmse: 1.30807
✅ Generated meta‐features for cell 2, shape = (328, 35)
Group 'oof': features=35, values=11480, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 3.58231
[200]	valid_0's rmse: 3.60743
Early stopping, best iteration is:
[56]	valid_0

In [12]:
import torch
from python_scripts.import_data import load_node_feature_data


image_keys = [ 'tile', 'subtiles']

model = VisionMLP_MultiTask(tile_dim=tile_dim, subtile_dim=center_dim, output_dim=C)

# 用法示例
from python_scripts.import_data import importDataset
# 假设你的 model 已经定义好并实例化为 `model`
test_dataset = load_node_feature_data("dataset/spot-rank/filtered_directly_rank/masked/test/Macenko/test_dataset.pt", model)
test_dataset = importDataset(
        data_dict=test_dataset,
        model=model,
        image_keys=image_keys,
        transform=lambda x: x,  # identity transform
        print_sig=True
    )



/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/import_data.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  raw = torch.load(pt_path, map_location="cpu")


⚠️ 從 '<class 'list'>' 推斷樣本數量: 2088
Model forward signature: (tile, subtiles)


In [ ]:
# --- 3) Prepare test meta-features ---
n_test = len(test_dataset)

all_final = []

for fold_id in range(n_folds):
    # if fold_id > start_fold:
    #     print(f"⏭️ Skipping fold {fold_id}")
    #     continue
    ckpt_path = os.path.join(trained_oof_model_folder, f"fold{fold_id}", "best_model.pt")
    net = VisionMLP_MultiTask(tile_dim=tile_dim, subtile_dim=center_dim, output_dim=C)
    net = net.to(device)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net.eval()

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_preds = []

    with torch.no_grad():
        for batch in test_loader:
            tiles = batch['tile'].to(device)
            subtiles = batch['subtiles'].to(device)
            center = subtiles[:, 4].contiguous()

            f_c = net.encoder_center(center)
            f_n = net.encoder_subtile(subtiles)
            f_t = net.encoder_tile(tiles)
            fuse = torch.cat([f_c, f_n, f_t], dim=1).contiguous()
            output = net.decoder(fuse)

            test_preds.append(output.cpu())
    test_preds = torch.cat(test_preds, dim=0).numpy()

    meta_model = joblib.load(
        os.path.join(trained_oof_model_folder,
                     f"fold{fold_id}", "meta_model.pkl")
    )
    resid_pred = np.zeros_like(test_preds)

    for i in range(C):
        # 2.1 生成第 i 类的“self vs others”特征
        meta_i, names_i = generate_meta_features(test_preds, i)
        # 简单检查
        diagnose_meta_nonfinite(meta_i, names_i)
        # 2.2 划分 train/val
        # 2.3 训练单输出 LGBM
        print(f"  → Cell {i}: meta-feat dim={meta_i.shape[1]}")
        resid_pred[:, i] = meta_model.estimators_[i].predict(meta_i)
        
    all_final.append(resid_pred)


final_preds = np.mean(all_final, axis=0)  # (n_test, C)

# --- Save submission ---
import h5py
import pandas as pd
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spot_ids = pd.DataFrame(np.array(f["spots/Test"]["S_7"]))
sub = pd.DataFrame(final_preds, columns=[f"C{i+1}" for i in range(C)])
sub.insert(0, 'ID', test_spot_ids.index)
sub.to_csv(os.path.join(trained_oof_model_folder, 'submission_stacked.csv'), index=False)
print(f"✅ Saved stacked submission in {trained_oof_model_folder}")


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_23908/3083716959.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt_path, ma

✅ Generated meta‐features for cell 0, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
✅ Generated meta‐features for cell 1, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
✅ Generated meta‐features for cell 2, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
✅ Generated meta‐features for cell 3, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 3: meta-feat dim=35
✅ Generated meta‐features for cell 4, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 4: meta-feat dim=35
✅ Generated meta‐features for cell 5, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 5: meta-feat dim=35
✅ Generated meta‐features for cell 6, shape = (2088, 35)
Group 'oof': featur

/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

✅ Generated meta‐features for cell 0, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
✅ Generated meta‐features for cell 1, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
✅ Generated meta‐features for cell 2, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
✅ Generated meta‐features for cell 3, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 3: meta-feat dim=35
✅ Generated meta‐features for cell 4, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 4: meta-feat dim=35
✅ Generated meta‐features for cell 5, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 5: meta-feat dim=35
✅ Generated meta‐features for cell 6, shape = (2088, 35)
Group 'oof': featur

/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

...

✅ Generated meta‐features for cell 0, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
✅ Generated meta‐features for cell 1, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
✅ Generated meta‐features for cell 2, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
✅ Generated meta‐features for cell 3, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 3: meta-feat dim=35
✅ Generated meta‐features for cell 4, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 4: meta-feat dim=35
✅ Generated meta‐features for cell 5, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 5: meta-feat dim=35
✅ Generated meta‐features for cell 6, shape = (2088, 35)
Group 'oof': featur

/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

✅ Generated meta‐features for cell 0, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 0: meta-feat dim=35
✅ Generated meta‐features for cell 1, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 1: meta-feat dim=35
✅ Generated meta‐features for cell 2, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 2: meta-feat dim=35
✅ Generated meta‐features for cell 3, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 3: meta-feat dim=35
✅ Generated meta‐features for cell 4, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 4: meta-feat dim=35
✅ Generated meta‐features for cell 5, shape = (2088, 35)
Group 'oof': features=35, values=73080, non-finite=0 (nan=0, inf=0)
  → Cell 5: meta-feat dim=35
✅ Generated meta‐features for cell 6, shape = (2088, 35)
Group 'oof': featur

/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

: 

In [ ]:
import glob
import torch
import numpy as np
import pandas as pd
import os
import h5py
from torch.utils.data import DataLoader

# 讀 test spot index
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spots     = f["spots/Test"]
    test_spot_table= pd.DataFrame(np.array(test_spots['S_7']))

fold_ckpts = sorted(glob.glob(os.path.join(trained_oof_model_folder, "fold*", "best_model.pt")))
models = []
for ckpt in fold_ckpts:
    net = VisionMLP_MultiTask(tile_dim=tile_dim, subtile_dim=center_dim, output_dim=C)
    net = net.to(device)
    net.load_state_dict(torch.load(ckpt, map_location="cpu"))
    net.to(device).eval()
    models.append(net)

all_fold_preds = []
for fold_id, net in enumerate(models):
    # 推論
    with torch.no_grad():
        preds = predict(net, test_loader, device)  # (N_test,35) numpy array

    # 1) 存每一折的原始預測
    df_fold = pd.DataFrame(preds, columns=[f"C{i+1}" for i in range(preds.shape[1])])
    df_fold.insert(0, "ID", test_spot_table.index)
    path_fold = os.path.join(trained_oof_model_folder, f"submission_fold{fold_id}.csv")
    df_fold.to_csv(path_fold, index=False)
    print(f"✅ Saved fold {fold_id} predictions to {path_fold}")

    all_fold_preds.append(preds)

# 2) 做 rank‐average ensemble
all_fold_preds = np.stack(all_fold_preds, axis=0)       # (K, N_test, 35)
ranks          = all_fold_preds.argsort(axis=2).argsort(axis=2).astype(float)
mean_rank      = ranks.mean(axis=0)                    # (N_test,35)

# 3) 存 final ensemble
df_ens = pd.DataFrame(mean_rank, columns=[f"C{i+1}" for i in range(mean_rank.shape[1])])
df_ens.insert(0, "ID", test_spot_table.index)
path_ens = os.path.join(trained_oof_model_folder, "submission_rank_ensemble.csv")
df_ens.to_csv(path_ens, index=False)
print(f"✅ Saved rank‐ensemble submission to {path_ens}")


array([[21.119944, 24.191807, 24.289158, ..., 12.108937,  9.812806,
        23.269012],
       [20.796225, 24.415634, 24.210833, ..., 12.384914,  9.888517,
        24.08245 ],
       [24.244806, 28.043358, 25.876398, ...,  9.660413,  9.817519,
        19.834955],
       ...,
       [21.064796, 30.208473, 23.31816 , ..., 11.452721,  8.848305,
        20.618204],
       [20.672852, 24.048859, 23.066084, ..., 12.829765, 10.746237,
        25.129263],
       [18.379812, 24.315762, 22.625013, ..., 12.282604, 10.867987,
        25.213802]], dtype=float32)

In [ ]:
sub

,ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
0,0,21.119944,24.191807,24.289158,12.459235,22.455460,22.283152,20.299072,10.036475,27.673294,...,11.457705,24.884144,17.928999,20.161509,2.243454,22.765985,23.942736,12.108937,9.812806,23.269012
1,1,20.796225,24.415634,24.210833,11.970597,20.137850,22.147005,20.423100,9.942406,27.705023,...,11.753860,25.524651,17.553988,20.109415,2.087860,21.848841,24.041878,12.384914,9.888517,24.082451
2,2,24.244806,28.043358,25.876398,15.009463,31.758268,16.058325,21.642458,9.751914,23.648655,...,8.703915,21.236656,9.903481,17.718140,2.080200,26.463696,23.671413,9.660413,9.817519,19.834955
3,3,21.303495,27.993895,23.471092,13.773635,32.076313,22.875986,20.917194,9.527581,27.025751,...,10.942430,26.616690,11.560891,19.593130,2.443240,23.056076,22.945707,10.667439,8.485572,20.935202
4,4,20.963795,26.275974,23.599735,12.649989,34.076267,18.544395,23.218969,9.845385,23.523705,...,10.565717,21.139025,14.137176,20.544558,2.245719,26.592495,24.027935,12.059454,10.051556,21.475361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2083,21.432768,25.581257,24.166622,12.927016,28.528118,20.753321,20.515493,10.021672,20.517944,...,11.892869,22.472292,14.630043,20.351019,3.042301,22.126217,24.394783,12.519063,10.259760,23.118910
2084,2084,21.026283,29.492777,24.808054,12.724218,33.597523,11.030184,22.632307,10.622198,20.301828,...,7.890387,17.251678,8.798381,20.905010,4.400732,27.635214,24.362818,11.217796,9.118665,21.300503
2085,2085,21.064796,30.208473,23.318159,12.652661,35.203560,12.808736,25.114325,9.819067,19.983475,...,7.644617,19.689543,8.118116,20.358732,2.374201,26.783195,23.938486,11.452721,8.848305,20.618204
2086,2086,20.672852,24.048859,23.066084,10.990246,13.805902,25.813622,19.609550,10.655597,30.124113,...,12.145758,28.294661,20.048624,18.838966,3.666246,19.281502,24.412367,12.829765,10.746237,25.129263


In [24]:
df_fold

,ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
0,0,21.918165,29.967756,22.074318,6.266089,15.045876,8.575323,17.701105,14.212624,14.427099,...,18.874729,8.440725,12.077477,10.421371,25.323563,25.987068,24.010574,13.736522,10.605394,18.971167
1,1,20.283955,31.262041,22.922802,4.601058,24.712463,3.007397,20.885874,13.523920,5.630240,...,18.687778,3.410803,11.689346,16.499268,22.862659,28.215260,25.980690,16.848274,11.301963,20.923363
2,2,31.330938,23.966707,30.072119,30.062901,30.986843,18.591246,18.741274,7.207420,20.441608,...,8.263720,16.578598,6.370902,10.274622,4.264868,23.713425,21.635761,9.099795,12.939436,18.898182
3,3,31.079088,24.034397,29.942924,29.694572,31.073133,18.107000,18.901016,7.263462,19.805269,...,8.317959,16.145647,6.501179,10.851070,4.138475,23.787277,21.747026,9.477861,13.075907,19.043671
4,4,17.650341,31.231380,22.025074,5.161528,35.091103,4.546699,23.162336,11.422981,6.613493,...,11.964938,6.922712,10.138801,26.204126,13.087643,28.410614,25.620949,17.164482,9.834173,20.661398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2083,30.925638,24.422085,29.964197,29.571100,31.709215,17.784489,19.358717,7.398783,19.437748,...,7.961905,16.280394,6.421072,11.637894,3.814150,24.170872,21.995071,9.395216,13.106522,19.251675
2084,2084,30.546535,25.253378,30.099052,26.397860,29.719896,12.094973,19.800642,8.228353,12.114425,...,11.054204,9.736452,6.732111,11.036299,4.972429,25.067688,23.207266,10.954745,15.503744,20.514071
2085,2085,23.831211,30.528559,26.471167,10.857256,29.458479,3.878749,21.609783,11.785152,4.007362,...,17.327324,3.710938,9.639662,18.194328,14.746003,28.351360,26.361732,17.171824,13.462471,21.873987
2086,2086,20.277044,30.499546,23.394804,13.227442,34.564075,10.764482,25.904028,10.293766,15.805928,...,5.561514,16.185980,7.185173,24.451096,2.226278,28.223160,25.268007,11.222231,9.121184,21.632837


In [ ]:
# --- 3) Prepare test meta-features ---
n_test = len(test_dataset)


for fold_id in range(n_folds):
    if fold_id > start_fold:
        print(f"⏭️ Skipping fold {fold_id}")
        continue
    ckpt_path = os.path.join(trained_oof_model_folder, f"fold{fold_id}", "best_model.pt")
    net = VisionMLP_MultiTask(tile_dim=tile_dim, subtile_dim=center_dim, output_dim=C)
    net = net.to(device)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net.eval()

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_preds = []
    test_latents = []

    with torch.no_grad():
        for batch in test_loader:
            tiles = batch['tile'].to(device)
            subtiles = batch['subtiles'].to(device)
            center = subtiles[:, 4].contiguous()

            f_c = net.encoder_center(center)
            f_n = net.encoder_subtile(subtiles)
            f_t = net.encoder_tile(tiles)
            fuse = torch.cat([f_c, f_n, f_t], dim=1).contiguous()
            output = net.decoder(fuse)

            test_preds.append(output.cpu())
            test_latents.append(fuse.cpu())


    test_preds = torch.cat(test_preds, dim=0).numpy()
    test_latents = torch.cat(test_latents, dim=0).numpy()
# === AE model reconstruction loss ===
    recon_model = PretrainedEncoderRegressor(
        ae_checkpoint=pretrained_ae_path,
        ae_type=ae_type,
        tile_dim=tile_dim,
        center_dim=center_dim,
        neighbor_dim=neighbor_dim,
        output_dim=C,
        mode='reconstruction'
    ).to(device)

    meta, name = generate_meta_features(
        dataset = test_dataset,
        oof_preds = test_preds,
        model_for_recon = recon_model,
        device = device,
        ae_type = ae_type,
    )
    # meta, name = groupwise_reduce(
    #     features=test_meta_initial,
    #     names=test_name_initial,
    # )
        
    # 1) 直接載入整個 MultiOutputRegressor
    meta_model_path = os.path.join(trained_oof_model_folder, f"fold{fold_id}","meta_model.pkl")
    meta_model = joblib.load(meta_model_path)

    # 2) 用剛剛算出的 meta features 做預測
    resid_pred = meta_model.predict(meta)
    final_preds = test_preds + resid_pred



# --- Save submission ---
import h5py
import pandas as pd
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spot_ids = pd.DataFrame(np.array(f["spots/Test"]["S_7"]))
sub = pd.DataFrame(final_preds, columns=[f"C{i+1}" for i in range(C)])
sub.insert(0, 'ID', test_spot_ids.index)
sub.to_csv(os.path.join(trained_oof_model_folder, 'submission_stacked.csv'), index=False)
print(f"✅ Saved stacked submission in {trained_oof_model_folder}")


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_3581/2907721651.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt_path, map

ae-recon-loss -> cols:    1, names:    1 OK
ae           -> cols:  384, names:  384 OK
subtile4     -> cols:   12, names:   12 OK
exsubtiles   -> cols:   12, names:   12 OK
tile         -> cols:   12, names:   12 OK
contrast     -> cols:    3, names:    3 OK
wavelet-tile -> cols:  280, names:  280 OK
oof          -> cols:   35, names:   35 OK
mad          -> cols:    1, names:    1 OK
✅ Generated meta-features with shape: (2088, 740)


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

⏭️ Skipping fold 1
⏭️ Skipping fold 2
⏭️ Skipping fold 3
⏭️ Skipping fold 4
⏭️ Skipping fold 5
✅ Saved stacked submission in output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [113]:
import numpy as np
import pandas as pd

# 1) 计算平均 feature_importances
importances = np.vstack([
    est.feature_importances_ 
    for est in meta_model.estimators_
]).mean(axis=0)

# 2) 构造 DataFrame 并排序
df_imp = pd.DataFrame({
    "feature": name,
    "importance": importances
})

# 2) 提取“类别”标签 —— 这里我们以第一个下划线前的字符串当作类别
df_imp['category'] = df_imp['feature'].apply(lambda x: x.split('_')[0])

# 3) 按类别聚合（求和或求平均都可以，下面示例用求和）
cat_imp = df_imp.groupby('category')['importance'] \
            .mean() \
            .sort_values(ascending=False)

print(cat_imp)


# 方法 A：临时设置最大行数，打印时不截断
pd.set_option('display.max_rows', df_imp.shape[0])
print(df_imp)
# 如果代码运行在脚本里，记得在打印完后恢复默认：
pd.reset_option('display.max_rows')

# 方法 B：直接 to_string()
print(df_imp.to_string(index=False))


category
sobel-tile         223.985714
sobel-subtile      204.554762
wavelet-subtile    200.403968
wavelet            194.213810
hsv-tile           178.371429
ae                 142.711310
he-tile            130.703571
oof                118.499592
hsv-subtile        116.662169
he-subtile         110.588889
ae-recon-loss       83.028571
exsubtiles          81.842857
tile                79.740476
subtile4            79.554762
wavelet-tile        71.435714
trained-latents     65.637277
contrast            58.733333
Name: importance, dtype: float64
                                  feature  importance         category
0                    ae-recon-loss_center   83.028571    ae-recon-loss
1                                 ae_emb0   59.771429               ae
2                                 ae_emb1   75.057143               ae
3                                 ae_emb2  106.942857               ae
4                                 ae_emb3   69.057143               ae
5                     

In [ ]:

# # Base model
# lgb_base = lgb.LGBMRegressor(
#     objective='l2',
#     metric='rmse',
#     n_estimators=12000,
#     max_depth=15,
#     learning_rate=0.008,
#     num_leaves=32,
#     colsample_bytree=0.25
# )

lgb_base = lgb.LGBMRegressor(
    objective='l2',
    metric='rmse',
    learning_rate=0.007522970004049377,
    n_estimators=12000,
    max_depth=11,
    num_leaves=194,
    colsample_bytree=0.7619407413363416,
    subsample=0.8,
    subsample_freq=1,
    min_data_in_leaf=20,
    reg_alpha=0.7480401395491829,
    reg_lambda=0.2589860348178542,
    verbosity=-1
)
# 將每個 target 分別 early stopping
meta_model = MultiOutputRegressor(lgb_base)

print("Training LightGBM on OOF meta-features with early stopping...")
meta_model.estimators_ = []

for i in range(y_train.shape[1]):
    print(f"Training target {i}...")
    model  = lgb.LGBMRegressor(
        objective='l2',
        metric='rmse',
        learning_rate=0.007522970004049377,
        n_estimators=12000,
        max_depth=11,
        num_leaves=194,
        colsample_bytree=0.7619407413363416,
        subsample=0.8,
        subsample_freq=1,
        min_data_in_leaf=20,
        reg_alpha=0.7480401395491829,
        reg_lambda=0.2589860348178542,
        verbosity=-1
    )

    model.fit(
        X_train,
        y_train[:, i],
        eval_set=[(X_val, y_val[:, i])],
        callbacks=[
            early_stopping(stopping_rounds=200),
            log_evaluation(period=100)
        ]
    )

    meta_model.estimators_.append(model)

# 保存模型
joblib.dump(meta_model, os.path.join(save_root, 'meta_model.pkl'))


# --- 3) Prepare test meta-features ---
n_test = len(test_dataset)
test_preds = []
test_latents = []

for fold_id in range(n_folds):
    ckpt_path = os.path.join(save_root, f"fold{fold_id}", "best_model.pt")
    net = PretrainedEncoderRegressor(
        ae_checkpoint=checkpoint_path,
        ae_type="all",
        center_dim=64, neighbor_dim=64, hidden_dim=128,
        tile_size=26, output_dim=35,
        freeze_encoder = True
    )

    net.decoder = nn.Sequential(
        nn.Linear(64+64, 256),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(256, 128),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(128, 64),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(64, 35)
    )

    net = net.to(device)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net.eval()

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    preds = []
    latents = []

    with torch.no_grad():
        for batch in test_loader:
            tiles = batch['tile'].to(device)
            subtiles = batch['subtiles'].to(device)

            center = subtiles[:, 4].contiguous()
            f_c = net.enc_center(center)
            f_n = net.enc_neigh(subtiles)
            fuse = torch.cat([f_c, f_n], dim=1)

            out = net.decoder(fuse)

            preds.append(out.cpu())
            latents.append(fuse.cpu())  # image embedding (128D)

    test_preds.append(torch.cat(preds, dim=0).numpy())      # shape: (n_test, 35)
    test_latents.append(torch.cat(latents, dim=0).numpy())  # shape: (n_test, 128)

# === Stack + Average ===
test_preds = np.mean(np.stack(test_preds, axis=0), axis=0)      # (n_test, 35)
test_latents = np.mean(np.stack(test_latents, axis=0), axis=0)  # (n_test, 128)

with h5py.File("dataset/elucidata_ai_challenge_data.h5", "r") as f:
    test_spots = f["spots/Test"]
    spot_array = np.array(test_spots['S_7'])
    df = pd.DataFrame(spot_array)

xy = df[["x", "y"]].to_numpy()  # shape: (n_test, 2)

# 合併為最終 test meta features
test_meta = np.concatenate([test_preds, xy, test_latents], axis=1)  # shape: (n_test, 35+2+128)



final_preds = meta_model.predict(test_meta)

# --- Save submission ---
import h5py
import pandas as pd
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spot_ids = pd.DataFrame(np.array(f["spots/Test"]["S_7"]))
sub = pd.DataFrame(final_preds, columns=[f"C{i+1}" for i in range(C)])
sub.insert(0, 'ID', test_spot_ids.index)
sub.to_csv(os.path.join(save_root, 'submission_stacked.csv'), index=False)
print(f"✅ Saved stacked submission in {save_root}")


In [ ]:
import os
import numpy as np
import joblib
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from scipy.stats import rankdata
from python_scripts.import_data import importDataset
from python_scripts.operate_model import predict
from lightgbm import early_stopping, log_evaluation
import h5py
import pandas as pd
# ---------------- Settings ----------------
save_root  = save_folder  # your save_folder path
n_folds    = len([d for d in os.listdir(save_root) if d.startswith('fold')])
n_samples  = len(full_dataset)
C          = 35  # num cell types
start_fold = 0
BATCH_SIZE = 64
# If optimizing Spearman, convert labels to ranks

# --- 1) Prepare OOF meta-features ---
# Initialize matrix for OOF predictions
n_samples = len(full_dataset)
oof_preds = np.zeros((n_samples, C), dtype=np.float32)
# True labels (raw or rank)
# importDataset returns a dict-like sample, so label is under key 'label'
y_true = np.vstack([ full_dataset[i]['label'].cpu().numpy() for i in range(n_samples) ])
y_meta = y_true

# Build CV splitter (must match first stage splits)
logo = LeaveOneGroupOut()
image_latents = np.zeros((n_samples, 128), dtype=np.float32)

# Loop over folds, load best model, predict on validation indices
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
for fold_id, (tr_idx, va_idx) in enumerate(
        logo.split(X=np.zeros(n_samples), y=None, groups=slide_idx)):
    # Load model
    # if fold_id > start_fold:
    #     print(f"⏭️ Skipping fold {fold_id}")
    #     continue
    ckpt_path = os.path.join(save_root, f"fold{fold_id}", "best_model.pt")
    print(f"Loading model from {ckpt_path}...")
    net = PretrainedEncoderRegressor(
        ae_checkpoint=checkpoint_path,
        ae_type="all",
        center_dim=64, neighbor_dim=64, hidden_dim=128,
        tile_size=26, output_dim=35,
        freeze_encoder = True
    )

    # 2) monkey‐patch 一个新的 head
    net.decoder  = nn.Sequential(
        nn.Linear(64+64, 256),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(256, 128),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(128, 64),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(64, 35)
        
    )
    net = net.to(device)    # Alternatively, if your model requires specific args, replace with:
    # net = VisionMLP_MultiTask(tile_dim=64, subtile_dim=64, output_dim=35).to(device)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net.to(device).eval()
    
    # Predict on validation set
    val_ds = Subset(full_dataset, va_idx)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

    preds = []
    latents = []

    with torch.no_grad():
        for batch in val_loader:
            tiles    = batch['tile'].to(device)
            subtiles = batch['subtiles'].to(device)

            center = subtiles[:, 4].contiguous()
            f_c = net.enc_center(center)
            f_n = net.enc_neigh(subtiles)
            fuse = torch.cat([f_c, f_n], dim=1)

            output = net.decoder(fuse)

            preds.append(output.cpu())
            latents.append(fuse.cpu())  # ⬅️ 收集 latent vector

    preds = torch.cat(preds, dim=0).numpy()    # (n_val, 35)
    latents = torch.cat(latents, dim=0).numpy()  # (n_val, 128)

    oof_preds[va_idx] = preds
    image_latents[va_idx] = latents

    print(f"Fold {fold_id}: OOF preds shape {preds.shape}, Latent shape: {latents.shape}")


    
with h5py.File("dataset/realign/filtered_dataset.h5", "r") as f:
    train_spots = f["spots/Train"]
    
    train_spot_tables = {}
    
    for slide_name in train_spots.keys():
        spot_array = np.array(train_spots[slide_name])
        df = pd.DataFrame(spot_array)
        df["slide_name"] = slide_name
        train_spot_tables[slide_name] = df
        print(f"✅ 已讀取 slide: {slide_name}")

# -----------------------------------------------------
# Step 2: 合併所有 slide 的資料
# -----------------------------------------------------
all_train_spots_df = pd.concat(train_spot_tables.values(), ignore_index=True)
# 提取 x, y
xy = all_train_spots_df[["x", "y"]].to_numpy()  # shape: (8348, 2)

# 合併成新的 meta feature
meta_features = np.concatenate([oof_preds, xy, image_latents], axis=1)
# --- 2) Train LightGBM meta-model ---
# Choose objective: regression on rank (for Spearman) or raw (for MSE)
# 將 meta features 拆成訓練集與 early stopping 用的驗證集
X_train, X_val, y_train, y_val = train_test_split(meta_features, y_meta, test_size=0.2, random_state=42)
print("Meta feature shape:", X_train.shape)
print("Feature std (min/max):", np.min(np.std(X_train, axis=0)), np.max(np.std(X_train, axis=0)))


# # Base model
# lgb_base = lgb.LGBMRegressor(
#     objective='l2',
#     metric='rmse',
#     n_estimators=12000,
#     max_depth=15,
#     learning_rate=0.008,
#     num_leaves=32,
#     colsample_bytree=0.25
# )
import optuna
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Define Optuna objective function
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'device': 'gpu',                # ✅ GPU 支援
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'learning_rate': trial.suggest_float("learning_rate", 0.005, 0.1),
        'max_depth': trial.suggest_int("max_depth", 4, 15),
        'num_leaves': trial.suggest_int("num_leaves", 32, 256),
        'min_data_in_leaf': trial.suggest_int("min_data_in_leaf", 20, 100),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 1.0),
        'reg_alpha': trial.suggest_float("reg_alpha", 0, 1),
        'reg_lambda': trial.suggest_float("reg_lambda", 0, 1),
        'n_estimators': 12000
    }

    model = lgb.LGBMRegressor(**params)
    multi_model = MultiOutputRegressor(model)
    multi_model.fit(X_train, y_train)

    y_pred = multi_model.predict(X_val)
    rmse = np.mean([
        np.sqrt(mean_squared_error(y_val[:, i], y_pred[:, i]))
        for i in range(y_val.shape[1])
    ])


    return rmse

# Run optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# Use best params to train final models
best_params = study.best_trial.params
best_params['objective'] = 'l2'
best_params['metric'] = 'rmse'
best_params['verbosity'] = -1

# Train final models with best parameters
meta_model = MultiOutputRegressor(lgb.LGBMRegressor(**best_params))
meta_model.estimators_ = []

print("Training LightGBM on OOF meta-features with best Optuna params...")
for i in range(y_train.shape[1]):
    print(f"Training target {i}...")
    model = lgb.LGBMRegressor(**best_params)

    model.fit(
        X_train,
        y_train[:, i],
        eval_set=[(X_val, y_val[:, i])],
        callbacks=[
            early_stopping(stopping_rounds=200),
            log_evaluation(period=100)
        ]
    )

    meta_model.estimators_.append(model)

# Save model
joblib.dump(meta_model, os.path.join(save_root, 'meta_model.pkl'))
# 保存模型


# --- 3) Prepare test meta-features ---
n_test = len(test_dataset)
test_preds = []
test_latents = []

for fold_id in range(n_folds):
    ckpt_path = os.path.join(save_root, f"fold{fold_id}", "best_model.pt")
    net = PretrainedEncoderRegressor(
        ae_checkpoint=checkpoint_path,
        ae_type="all",
        center_dim=64, neighbor_dim=64, hidden_dim=128,
        tile_size=26, output_dim=35,
        freeze_encoder = True
    )

    net.decoder = nn.Sequential(
        nn.Linear(64+64, 256),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(256, 128),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(128, 64),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(64, 35)
    )

    net = net.to(device)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net.eval()

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    preds = []
    latents = []

    with torch.no_grad():
        for batch in test_loader:
            tiles = batch['tile'].to(device)
            subtiles = batch['subtiles'].to(device)

            center = subtiles[:, 4].contiguous()
            f_c = net.enc_center(center)
            f_n = net.enc_neigh(subtiles)
            fuse = torch.cat([f_c, f_n], dim=1)

            out = net.decoder(fuse)

            preds.append(out.cpu())
            latents.append(fuse.cpu())  # image embedding (128D)

    test_preds.append(torch.cat(preds, dim=0).numpy())      # shape: (n_test, 35)
    test_latents.append(torch.cat(latents, dim=0).numpy())  # shape: (n_test, 128)

# === Stack + Average ===
test_preds = np.mean(np.stack(test_preds, axis=0), axis=0)      # (n_test, 35)
test_latents = np.mean(np.stack(test_latents, axis=0), axis=0)  # (n_test, 128)

with h5py.File("dataset/elucidata_ai_challenge_data.h5", "r") as f:
    test_spots = f["spots/Test"]
    spot_array = np.array(test_spots['S_7'])
    df = pd.DataFrame(spot_array)

xy = df[["x", "y"]].to_numpy()  # shape: (n_test, 2)

# 合併為最終 test meta features
test_meta = np.concatenate([test_preds, xy, test_latents], axis=1)  # shape: (n_test, 35+2+128)



final_preds = meta_model.predict(test_meta)

# --- Save submission ---
import h5py
import pandas as pd
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spot_ids = pd.DataFrame(np.array(f["spots/Test"]["S_7"]))
sub = pd.DataFrame(final_preds, columns=[f"C{i+1}" for i in range(C)])
sub.insert(0, 'ID', test_spot_ids.index)
sub.to_csv(os.path.join(save_root, 'submission_stacked.csv'), index=False)
print("✅ Saved stacked submission.")


In [ ]:
import os
import numpy as np
import joblib
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
from scipy.stats import rankdata
from python_scripts.import_data import importDataset
from python_scripts.operate_model import predict

# --- 配置: 只用哪些 fold 的结果来训练/预测 meta-model ---
meta_folds = [0]  # 例如只用 fold0, fold2, fold4

# 1) 准备 full_dataset, slide_idx, test_dataset 等
full_dataset = importDataset(
    grouped_data, model,
    image_keys=['tile','subtiles'],
    transform=lambda x: x
)
n_samples = len(full_dataset)
C = 35  # 类别数

# 2) 预留 oof_preds 和 fold_ids
oof_preds    = np.zeros((n_samples, C), dtype=np.float32)
oof_fold_ids = np.full(n_samples, -1, dtype=int)

# 真标签
y_true = np.vstack([ full_dataset[i]['label'].cpu().numpy() for i in range(n_samples) ])
y_meta = y_true.copy()  # 不做 rank 时直接用 raw

# 3) 生成 OOF 预测并记录 fold id
logo = LeaveOneGroupOut()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

for fold_id, (tr_idx, va_idx) in enumerate(
        logo.split(X=np.zeros(n_samples), y=None, groups=slide_idx)):

    # 如果当前 fold 不在我们想要的 meta_folds 列表里，就跳过
    if fold_id not in meta_folds:
        print(f"⏭️ Skipping OOF for fold {fold_id}")
        continue

    print(f"\n>>> Generating OOF for fold {fold_id}")
    ckpt_path = os.path.join(save_root, f"fold{fold_id}", "best_model.pt")
    net = PretrainedEncoderRegressor(
        ae_checkpoint=checkpoint_path,
        ae_type="all",
        center_dim=64, neighbor_dim=64, hidden_dim=128,
        tile_size=26, output_dim=35,
        freeze_encoder = True
    )

    # 2) monkey‐patch 一个新的 head
    net.decoder  = nn.Sequential(
        nn.Linear(64+64, 128),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(128, 64),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(64, 35)
        
    )
    net = net.to(device)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net.eval()

    val_loader = DataLoader(Subset(full_dataset, va_idx), batch_size=BATCH_SIZE, shuffle=False)
    preds = predict(net, val_loader, device)  # (n_val, C)

    oof_preds[va_idx]    = preds
    oof_fold_ids[va_idx] = fold_id

    print(f"  → Fold {fold_id} OOF preds shape: {preds.shape}")
# 4) 只选取 meta_folds 的行来训练 meta-model
mask = np.isin(oof_fold_ids, meta_folds)
X_meta = oof_preds[mask]
y_meta_sub = y_meta[mask]

print(f"\nTraining meta-model on folds {meta_folds}:")
print(f"  使用样本数：{X_meta.shape[0]} / {n_samples}")

lgb_base = lgb.LGBMRegressor(
    objective='regression',
    learning_rate=0.001,
    n_estimators=1000,
    num_leaves=31,
    subsample=0.7,
    colsample_bytree=0.7,
    n_jobs=-1,
    force_col_wise=True
)
meta_model = MultiOutputRegressor(lgb_base)
meta_model.fit(X_meta, y_meta_sub)
joblib.dump(meta_model, os.path.join(save_root, 'meta_model.pkl'))

# 5) 准备 test_meta，只平均 meta_folds 中的预测
n_folds = len([d for d in os.listdir(save_root) if d.startswith('fold')])
n_test  = len(test_dataset)
test_meta = np.zeros((n_test, C), dtype=np.float32)

for fold_id in range(n_folds):
    if fold_id not in meta_folds:
        continue
    ckpt_path = os.path.join(save_root, f"fold{fold_id}", "best_model.pt")
    net = PretrainedEncoderRegressor(
        ae_checkpoint=checkpoint_path,
        ae_type="all",
        center_dim=64, neighbor_dim=64, hidden_dim=128,
        tile_size=26, output_dim=35,
        freeze_encoder = True
    )

    # 2) monkey‐patch 一个新的 head
    net.decoder  = nn.Sequential(
        nn.Linear(64+64, 128),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(128, 64),
        nn.SiLU(),
        nn.Dropout(0.1),
        nn.Linear(64, 35)
        
    )
    net = net.to(device)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net.eval()

    loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    preds = predict(net, loader, device)
    test_meta += preds

# 平均时除以参与的 folds 数目
test_meta /= len(meta_folds)

# 6) 用 meta-model 做最终预测
final_preds = meta_model.predict(test_meta)

# --- Save submission ---
import h5py
import pandas as pd

with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spot_ids = pd.DataFrame(np.array(f["spots/Test"]["S_7"]))

sub = pd.DataFrame(final_preds, columns=[f"C{i+1}" for i in range(C)])
sub.insert(0, 'ID', test_spot_ids.index)
sub.to_csv(os.path.join(save_root, 'submission_stacked.csv'), index=False)
print("✅ Saved stacked submission.")


# Predict

In [21]:
import glob
import torch
import numpy as np
import pandas as pd
import os
import h5py
from torch.utils.data import DataLoader

# 讀 test spot index
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spots     = f["spots/Test"]
    test_spot_table= pd.DataFrame(np.array(test_spots['S_7']))

fold_ckpts = sorted(glob.glob(os.path.join(trained_oof_model_folder, "fold*", "best_model.pt")))
models = []
for ckpt in fold_ckpts:
    net = VisionMLP_MultiTask(tile_dim=tile_dim, subtile_dim=center_dim, output_dim=C)
    net = net.to(device)
    net.load_state_dict(torch.load(ckpt, map_location="cpu"))
    net.to(device).eval()
    models.append(net)

all_fold_preds = []
for fold_id, net in enumerate(models):
    # 推論
    with torch.no_grad():
        preds = predict(net, test_loader, device)  # (N_test,35) numpy array

    # 1) 存每一折的原始預測
    df_fold = pd.DataFrame(preds, columns=[f"C{i+1}" for i in range(preds.shape[1])])
    df_fold.insert(0, "ID", test_spot_table.index)
    path_fold = os.path.join(trained_oof_model_folder, f"submission_fold{fold_id}.csv")
    df_fold.to_csv(path_fold, index=False)
    print(f"✅ Saved fold {fold_id} predictions to {path_fold}")

    all_fold_preds.append(preds)

# 2) 做 rank‐average ensemble
all_fold_preds = np.stack(all_fold_preds, axis=0)       # (K, N_test, 35)
ranks          = all_fold_preds.argsort(axis=2).argsort(axis=2).astype(float)
mean_rank      = ranks.mean(axis=0)                    # (N_test,35)

# 3) 存 final ensemble
df_ens = pd.DataFrame(mean_rank, columns=[f"C{i+1}" for i in range(mean_rank.shape[1])])
df_ens.insert(0, "ID", test_spot_table.index)
path_ens = os.path.join(trained_oof_model_folder, "submission_rank_ensemble.csv")
df_ens.to_csv(path_ens, index=False)
print(f"✅ Saved rank‐ensemble submission to {path_ens}")


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_23908/317847752.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt, map_loca

✅ Saved fold 0 predictions to output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/submission_fold0.csv
✅ Saved fold 1 predictions to output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/submission_fold1.csv
✅ Saved fold 2 predictions to output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/submission_fold2.csv
✅ Saved fold 3 predictions to output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/submission_fold3.csv
✅ Saved fold 4 predictions to output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/submission_fold4.csv
✅ Saved fold 5 predictions to output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/submission_fold5.csv
✅ Saved rank‐ensemble submis